


# Assignment 5 on Natural Language Processing

## Date : 3rd Nov, 2020

### Instructor : Prof. Sudeshna Sarkar

### Teaching Assistants : Alapan Kuila, Aniruddha Roy, Anusha Potnuru, Uppada Vishnu

The central idea of this assignment is to explore various language models specifically LSTM based and transformer. We will explore how the size of the model effects the sequence generated. We will see both character based and word based models.



Please submit with outputs. Submissions without predicted outputs will be penalized.

# Word Based LSTM model

In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import numpy
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras

Do basic pre processing which includes lowering etc
Check the dataset and apply suitable preprocessing.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Load the data and preprocess data and store corpus in raw_text
with open(file='/content/drive/My Drive/Colab Notebooks/NLP/NLP Assignment-5/corpus.txt', mode='r', encoding="UTF8") as file:
    corpus = file.read().replace('\n', '')

import nltk
import unicodedata
nltk.download('punkt')
from nltk import word_tokenize, sent_tokenize

"""Getting sentences from the corpus"""
sents = nltk.sent_tokenize(corpus)

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation and _ from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '' and new_word != '_':
            if new_word[0] == '_':
                  new_word = new_word[1:]
            if new_word[-1] == '_':
                new_word = new_word[:-2]
            new_words.append(new_word) 
    return new_words

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('UTF8', 'ignore')
        new_words.append(new_word)
    return new_words

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    return words

preprocessed_sents = []
for sent in sents:
    words = nltk.word_tokenize(sent)
    words = normalize(words)
    new_sent = " ".join(words)
    preprocessed_sents.append(new_sent)

raw_text = ""
for sent in preprocessed_sents:
    raw_text = raw_text + " " + sent

raw_text = raw_text.strip()
raw_text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


'chapter idown the rabbithole alice was beginning to get very tired of sitting by her sister on the bank and of having nothing to do once or twice she had peeped into the book her sister was reading but it had no pictures or conversations in it and what is the use of a book thought alice without pictures or conversations so she was considering in her own mind as well as she could for the hot day made her feel very sleepy and stupid whether the pleasure of making a daisychain would be worth the trouble of getting up and picking the daisies when suddenly a white rabbit with pink eyes ran close by herthere was nothing so ver remarkable in that nor did alice think it so ver much out of the way to hear the rabbit say to itself oh dear oh dear i shall be late when she thought it over afterwards it occurred to her that she ought to have wondered at this but at the time it all seemed quite natural but when the rabbit actually took a watch out of its waistcoatpocke and looked at it and then hur

In [4]:
# Hyperparameters of the model
vocab_size = 2700 # choose based on statistics
oov_tok = '<OOV>'
embedding_dim = 100
padding_type='post'
trunc_type='post'

In [5]:
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts([raw_text])
word_index = tokenizer.word_index

In [6]:
seq_length = 50
tokens = tokenizer.texts_to_sequences([raw_text])[0]

In [7]:
dataX = []
dataY = []

for i in range(0, len(tokens) - seq_length-1 , 1):
  seq_in = tokens[i:i + seq_length]
  seq_out = tokens[i + seq_length]

  if seq_out==1: #Skip samples where target word is OOV
    continue
    
  dataX.append(seq_in)
  dataY.append(seq_out)
 
N = len(dataX)
print ("Total training data size: ", N)

Total training data size:  26375


In [8]:
X = numpy.array(dataX)

# one hot encode the output variable
y = numpy.array(dataY)
y = np_utils.to_categorical(dataY)

In [9]:
# with embedding
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=seq_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(vocab_size, activation='softmax')
])
# compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 100)           270000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               84480     
_________________________________________________________________
dense (Dense)                (None, 2700)              348300    
Total params: 702,780
Trainable params: 702,780
Non-trainable params: 0
_________________________________________________________________


In [10]:
print(X.shape)
print(y.shape)

(26375, 50)
(26375, 2700)


In [11]:
# Use validation split of 0.2 while training
model.fit(X, y, epochs=100, batch_size=128, validation_split=0.2) 

Epoch 1/100
165/165 [==============================] - 7s 44ms/step - loss: 6.4406 - accuracy: 0.0521 - val_loss: 6.0213 - val_accuracy: 0.0861
Epoch 2/100
165/165 [==============================] - 6s 39ms/step - loss: 5.9697 - accuracy: 0.0533 - val_loss: 5.9266 - val_accuracy: 0.0861
Epoch 3/100
165/165 [==============================] - 6s 39ms/step - loss: 5.8523 - accuracy: 0.0552 - val_loss: 5.9001 - val_accuracy: 0.0878
Epoch 4/100
165/165 [==============================] - 6s 39ms/step - loss: 5.7492 - accuracy: 0.0629 - val_loss: 5.8578 - val_accuracy: 0.0936
Epoch 5/100
165/165 [==============================] - 6s 39ms/step - loss: 5.6475 - accuracy: 0.0662 - val_loss: 5.8341 - val_accuracy: 0.0950
Epoch 6/100
165/165 [==============================] - 6s 39ms/step - loss: 5.5514 - accuracy: 0.0736 - val_loss: 5.7952 - val_accuracy: 0.1024
Epoch 7/100
165/165 [==============================] - 7s 40ms/step - loss: 5.4425 - accuracy: 0.0862 - val_loss: 5.7731 - val_accuracy:

In [12]:
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

In [13]:
# Complete the code to return next n words greedily
def next_tokens(input_str, n): 
	final_string = input_str
	print ("Seed: \n",  input_str)
	for i in range(n):
		token_list = tokenizer.texts_to_sequences([input_str])[0]
		token_list = pad_sequences([token_list], maxlen=seq_length-1, padding='pre', truncating='pre')
		prediction = model.predict_classes(token_list , verbose=0)
		output_word = ""
		for word, index in tokenizer.word_index.items():
			if index == prediction:
				output_word = word
				break
		input_str += " " + output_word
		final_string += " " + output_word
		# get next word index. Use reverse_word_map to get the word
	return final_string

In [14]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ' '.join([reverse_word_map[value] for value in pattern])

print(next_tokens(input_str , 10))

Seed: 
 i wonder what they wil do next if they had any sense they d take the roof off after a minute or two they began moving about again and alice heard the rabbit say a barrowful will do to begin with a barrowful of what thought alice but she had
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
i wonder what they wil do next if they had any sense they d take the roof off after a minute or two they began moving about again and alice heard the rabbit say a barrowful will do to begin with a barrowful of what thought alice but she had been a little more table how can t be a


In [15]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

# Use first 50 tokens from given input_str as input.(Use tokenizer to split to take first 50)
print(next_tokens(" ".join(word_tokenize(input_str)[:50]) , 10))

Seed: 
 The boy laughed at the fright he had caused . This time , the villagers left angrily . The third day , as the boy went up the small hill , he suddenly saw a wolf attacking his sheep . He cried as hard as he could , “ Wolf
The boy laughed at the fright he had caused . This time , the villagers left angrily . The third day , as the boy went up the small hill , he suddenly saw a wolf attacking his sheep . He cried as hard as he could , “ Wolf i should like the next thing is the mouse s


# Character based LSTM Model 1

In [16]:
# User the preprocess data and create raw_text
# Load the data and preprocess data and store corpus in raw_text
with open(file='/content/drive/My Drive/Colab Notebooks/NLP/NLP Assignment-5/corpus.txt', mode='r', encoding="UTF8") as file:
    corpus = file.read().replace('\n', '')

import nltk
import unicodedata
nltk.download('punkt')
from nltk import word_tokenize, sent_tokenize

"""Getting sentences from the corpus"""
sents = nltk.sent_tokenize(corpus)

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation and _ from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'["""[""", "(", ")", ":", ";", "_"]', '', word)
        new_word = re.sub("]", '', new_word)
        if new_word != '' and new_word != '_':
            if new_word[0] == '_':
                  new_word = new_word[1:]
            if new_word[-1] == '_':
                new_word = new_word[:-2]
            new_words.append(new_word) 
    return new_words

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('UTF8', 'ignore')
        new_words.append(new_word)
    return new_words

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    return words

preprocessed_sents = []
for sent in sents:
    words = nltk.word_tokenize(sent)
    words = normalize(words)
    new_sent = " ".join(words)
    preprocessed_sents.append(new_sent)

raw_text = ""
for sent in preprocessed_sents:
    raw_text = raw_text + " " + sent

raw_text = raw_text.strip()

# create mapping of unique characters to integers
chars = sorted(list(set(raw_text)))

char_to_int = dict([(y,x+1) for x,y in enumerate(chars)])
char_to_int

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


{' ': 1,
 '!': 2,
 '-': 3,
 '.': 4,
 '?': 5,
 'a': 6,
 'b': 7,
 'c': 8,
 'd': 9,
 'e': 10,
 'f': 11,
 'g': 12,
 'h': 13,
 'i': 14,
 'j': 15,
 'k': 16,
 'l': 17,
 'm': 18,
 'n': 19,
 'o': 20,
 'p': 21,
 'q': 22,
 'r': 23,
 's': 24,
 't': 25,
 'u': 26,
 'v': 27,
 'w': 28,
 'x': 29,
 'y': 30,
 'z': 31}

In [17]:
# Print the total characters and character vacob size
n_chars = len(list(raw_text))
n_vocab = len(char_to_int)
print(n_chars, n_vocab)

136860 31


In [18]:
tokens = []
for item in list(raw_text):
  tokens.append(char_to_int[item])
print(tokens)

[8, 13, 6, 21, 25, 10, 23, 1, 14, 4, 9, 20, 28, 19, 1, 25, 13, 10, 1, 23, 6, 7, 7, 14, 25, 3, 13, 20, 17, 10, 1, 6, 17, 14, 8, 10, 1, 28, 6, 24, 1, 7, 10, 12, 14, 19, 19, 14, 19, 12, 1, 25, 20, 1, 12, 10, 25, 1, 27, 10, 23, 30, 1, 25, 14, 23, 10, 9, 1, 20, 11, 1, 24, 14, 25, 25, 14, 19, 12, 1, 7, 30, 1, 13, 10, 23, 1, 24, 14, 24, 25, 10, 23, 1, 20, 19, 1, 25, 13, 10, 1, 7, 6, 19, 16, 1, 6, 19, 9, 1, 20, 11, 1, 13, 6, 27, 14, 19, 12, 1, 19, 20, 25, 13, 14, 19, 12, 1, 25, 20, 1, 9, 20, 1, 20, 19, 8, 10, 1, 20, 23, 1, 25, 28, 14, 8, 10, 1, 24, 13, 10, 1, 13, 6, 9, 1, 21, 10, 10, 21, 10, 9, 1, 14, 19, 25, 20, 1, 25, 13, 10, 1, 7, 20, 20, 16, 1, 13, 10, 23, 1, 24, 14, 24, 25, 10, 23, 1, 28, 6, 24, 1, 23, 10, 6, 9, 14, 19, 12, 1, 7, 26, 25, 1, 14, 25, 1, 13, 6, 9, 1, 19, 20, 1, 21, 14, 8, 25, 26, 23, 10, 24, 1, 20, 23, 1, 8, 20, 19, 27, 10, 23, 24, 6, 25, 14, 20, 19, 24, 1, 14, 19, 1, 14, 25, 1, 6, 19, 9, 1, 28, 13, 6, 25, 1, 14, 24, 1, 25, 13, 10, 1, 26, 24, 10, 1, 20, 11, 1, 6, 1, 7, 20, 2

In [19]:
'''
Prepare dataset where the input is sequence of 100 characters and target is next character.
'''
seq_length = 100
dataX = []
dataY = []
for i in range(0, len(tokens) - seq_length-1 , 1):
    # Write code here
  seq_in = tokens[i:i + seq_length]
  seq_out = tokens[i + seq_length]

  dataX.append(seq_in)
  dataY.append(seq_out)

n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  136759


In [20]:
# reshape X to be [samples, time steps, features]
X = numpy.array(dataX)

# one hot encode the output variable
dataY = numpy.array(dataY)
y = np_utils.to_categorical(dataY)

In [21]:
print(X.shape)
print(y.shape)

(136759, 100)
(136759, 32)


In [22]:
X[0].shape

(100,)

In [23]:
embedding_dim =100
max_length =100

In [24]:
model = Sequential()
model.add(Embedding(32, embedding_dim, input_length=max_length))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          3200      
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               365568    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                8224      
Total params: 376,992
Trainable params: 376,992
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.fit(X, y, epochs=20, batch_size=128, validation_split=0.2)

Epoch 1/20
855/855 [==============================] - 40s 47ms/step - loss: 2.2012 - val_loss: 1.8834
Epoch 2/20
855/855 [==============================] - 39s 46ms/step - loss: 1.7378 - val_loss: 1.6490
Epoch 3/20
855/855 [==============================] - 39s 46ms/step - loss: 1.5385 - val_loss: 1.5238
Epoch 4/20
855/855 [==============================] - 39s 45ms/step - loss: 1.4201 - val_loss: 1.4555
Epoch 5/20
855/855 [==============================] - 39s 45ms/step - loss: 1.3339 - val_loss: 1.4233
Epoch 6/20
855/855 [==============================] - 39s 46ms/step - loss: 1.2696 - val_loss: 1.3969
Epoch 7/20
855/855 [==============================] - 39s 46ms/step - loss: 1.2143 - val_loss: 1.3869
Epoch 8/20
855/855 [==============================] - 39s 46ms/step - loss: 1.1714 - val_loss: 1.3789
Epoch 9/20
855/855 [==============================] - 39s 46ms/step - loss: 1.1274 - val_loss: 1.3812
Epoch 10/20
855/855 [==============================] - 39s 45ms/step - loss: 1.086

In [26]:
#implement mapping of integer to character
int_to_char =dict([(x+1,y) for x,y in enumerate(chars)])
int_to_char

{1: ' ',
 2: '!',
 3: '-',
 4: '.',
 5: '?',
 6: 'a',
 7: 'b',
 8: 'c',
 9: 'd',
 10: 'e',
 11: 'f',
 12: 'g',
 13: 'h',
 14: 'i',
 15: 'j',
 16: 'k',
 17: 'l',
 18: 'm',
 19: 'n',
 20: 'o',
 21: 'p',
 22: 'q',
 23: 'r',
 24: 's',
 25: 't',
 26: 'u',
 27: 'v',
 28: 'w',
 29: 'x',
 30: 'y',
 31: 'z'}

In [27]:
'''
Complete code below to get the generated string using the model.
'''
def predict_next_100_chars(pattern,x):
	final_string  = pattern
	for i in range(x):
		 token_list = []
		 for item in list(pattern):
			   token_list.append(char_to_int[item])
		 token_list = numpy.array(token_list).reshape(1, 100)
		 prediction = int_to_char[model.predict_classes(token_list , verbose=0)[0]]
		 final_string += prediction
		 pattern += prediction
		 pattern = pattern[1:]
	return final_string

In [28]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ''.join([int_to_char[value] for value in pattern])
print(predict_next_100_chars(input_str, 200))

h puzzled her a good deal until she made out what it was she was beginning to grow larger again and see what was the baby it was an old croquet she was going to be a little startled with a sudden but it was an old crabbod about it and the moral of that is the march hare went on in a moment that it w


In [29]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

 # Use first 100 characeters from given input_str as input and generate next 200 characters.
sents = nltk.sent_tokenize(input_str)
preprocessed_sents = []
for sent in sents:
    words = nltk.word_tokenize(sent)
    words = normalize(words)
    new_sent = " ".join(words)
    preprocessed_sents.append(new_sent)

input_str = ""
for sent in preprocessed_sents:
    input_str = input_str + " " + sent

input_str = input_str.strip()


print(predict_next_100_chars(input_str[:100],200))

the boy laughed at the fright he had caused . this time the villagers left angrily . the third day and said to alice and she said to herself as she said the mock turtle said and the moral of that is the march hare went on in a moment that it was a great deal of that ? in a minute or two which was a 


## Character based LSTM Model 2


In [30]:
model1 = Sequential()
model1.add(Embedding(32, embedding_dim, input_length=max_length))
model1.add(LSTM(256, input_shape=(X.shape[1], embedding_dim),return_sequences=True))
model1.add(Dropout(0.2))
model1.add(LSTM(256))
model1.add(Dropout(0.2))
model1.add(Dense(y.shape[1], activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam')

In [31]:
model1.fit(X, y, epochs=20, batch_size=64)

Epoch 1/20
2137/2137 [==============================] - 142s 67ms/step - loss: 2.0406
Epoch 2/20
2137/2137 [==============================] - 143s 67ms/step - loss: 1.5321
Epoch 3/20
2137/2137 [==============================] - 144s 67ms/step - loss: 1.3662
Epoch 4/20
2137/2137 [==============================] - 143s 67ms/step - loss: 1.2759
Epoch 5/20
2137/2137 [==============================] - 143s 67ms/step - loss: 1.2125
Epoch 6/20
2137/2137 [==============================] - 141s 66ms/step - loss: 1.1634
Epoch 7/20
2137/2137 [==============================] - 141s 66ms/step - loss: 1.1209
Epoch 8/20
2137/2137 [==============================] - 141s 66ms/step - loss: 1.0853
Epoch 9/20
2137/2137 [==============================] - 141s 66ms/step - loss: 1.0558
Epoch 10/20
2137/2137 [==============================] - 142s 66ms/step - loss: 1.0289
Epoch 11/20
2137/2137 [==============================] - 142s 67ms/step - loss: 1.0034
Epoch 12/20
2137/2137 [=============================

In [32]:
'''
Complete code below to get the generated string using the model.
'''
def predict_next_100_chars(pattern,x):
	final_string  = pattern
	for i in range(x):
		 token_list = []
		 for item in list(pattern):
			   token_list.append(char_to_int[item])
		 token_list = numpy.array(token_list).reshape(1, 100)
		 prediction = int_to_char[model.predict_classes(token_list , verbose=0)[0]]
		 final_string += prediction
		 pattern += prediction
		 pattern = pattern[1:]
	return final_string

In [33]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ''.join([int_to_char[value] for value in pattern])

print(predict_next_100_chars(input_str,200))

ined that they could not taste theirs and the small ones choked and had to be patted on the back.however she said the dormouse shook itself and was going to see it was a great deal of that is the same thing said the dormouse s argument was that it was an old crabbod about it and the moral of that is


In [35]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

 # Use first 100 characeters from given input_str as input and generate next 200 characters.
sents = nltk.sent_tokenize(input_str)
preprocessed_sents = []
for sent in sents:
    words = nltk.word_tokenize(sent)
    words = normalize(words)
    new_sent = " ".join(words)
    preprocessed_sents.append(new_sent)

input_str = ""
for sent in preprocessed_sents:
    input_str = input_str + " " + sent

input_str = input_str.strip()
 
print(predict_next_100_chars(input_str[:100],200))

the boy laughed at the fright he had caused . this time the villagers left angrily . the third day and said to alice and she said to herself as she said the mock turtle said and the moral of that is the march hare went on in a moment that it was a great deal of that ? in a minute or two which was a 


# Questions

**Question:** What are your observations based on the model(all) outputs on train data(in domain) vs unseen data(out of domain) ?

**Answer:** Word and Chatacter based models are predicting next words and characters respectively but prediction is really bad in all the three cases on unseen dataset. In word based the model is clearly overfitting because training accuracy is very high and validation accuracy is very less. We may add some dropout layers to avoid the overfitting.

**Question:** What did you observe in the outputs of char LSTM model1 vs char LSTM model2 ?

**Answer:** On the basis of predictions none of them are good, but in LSTM2 we are adding 2 LSTM layers and 2 Dropout layers so it will not be overfitting and also will be predicting a bit better than LSTM1.  

# Bonus (Not to be graded)

## Transformer based language model (Bert)
You can run the below cells to predict the next word using pretrained BERT model. 

In [ ]:
!pip install transformers

In [ ]:
import os
import torch
import string
from transformers import BertTokenizer, BertForMaskedLM

In [ ]:
def load_model(model_name):
  try:
    if model_name.lower() == "bert":
      bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
      bert_model = BertForMaskedLM.from_pretrained('bert-base-uncased').eval()
      return bert_tokenizer,bert_model
  except Exception as e:
    pass

In [ ]:
def decode(tokenizer, pred_idx, top_clean):
  ignore_tokens = string.punctuation + '[PAD]'
  tokens = []
  for w in pred_idx:
    token = ''.join(tokenizer.decode(w).split())
    if token not in ignore_tokens:
      tokens.append(token.replace('##', ''))
  return '\n'.join(tokens[:top_clean])

In [ ]:
def encode(tokenizer, text_sentence, add_special_tokens=True):
  text_sentence = text_sentence.replace('<mask>', tokenizer.mask_token)
  # if <mask> is the last token, append a "." so that models dont predict punctuation.
  if tokenizer.mask_token == text_sentence.split()[-1]:
    text_sentence += ' .'
  input_ids = torch.tensor([tokenizer.encode(text_sentence, add_special_tokens=add_special_tokens)])
  mask_idx = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
  return input_ids, mask_idx

In [ ]:
def get_all_predictions(text_sentence, top_clean=5):
  input_ids, mask_idx = encode(bert_tokenizer, text_sentence)
  with torch.no_grad():
    predict = bert_model(input_ids)[0]
    print(predict.shape)
    
    bert = decode(bert_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)
  return {'bert': bert}

In [ ]:
def get_prediction_eos(input_text):
  try:
    input_text += ' <mask>'
    res = get_all_predictions(input_text, top_clean=int(top_k))
    return res
  except Exception as error:
    pass

In [ ]:
# Below code predicts the next top_k words. You can modify this to get next n words using top_k=1 and greedy decoding it. 
top_k= 3
print('Predict next ', top_k, ' words')
model_name = 'BERT'
bert_tokenizer, bert_model  = load_model(model_name) 
input_text = "" ### GIVE YOUR INPUT STRING HERE
res = get_prediction_eos(input_text)
answer = []
print(res['bert'].split("\n"))
for i in res['bert'].split("\n"):
  answer.append(i)
  answer_as_string = "    ".join(answer)

print(answer_as_string)